In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from datetime import datetime
from dateutil import parser

import uuid

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime
from dateutil import parser
import time
from urllib import request
import csv
import re


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [3]:
# Pour DL Chromedriver  : # Pour DL Chromedriver  : https://sites.google.com/a/chromium.org/chromedriver/

In [4]:
attributs_labels = []
games_list = []
attributs_labels_players = []
teams_stats_list = []
players_stats_list = []
link_pages = []

max_players = 1000000000
# Environ pour 20 ans(?)
max_games = 28000
# max_games = 15

In [5]:
def SaveInCsv(data):
    games = []
  
    for item in data:    
        games.append({
            'game_id': item.get('game_id'),
            'date_game': item.get('date_game'),
            'game_start_time': item.get('game_start_time'),
            'v_team_name': item.get('v_team_name'),
            'v_pts': item.get('v_pts'),
            'v_record': item.get('v_record'),
            'v_field_goals': item.get('v_field_goals'),
            'v_field_goal_attempts': item.get('v_field_goal_attempts'),
            'v_field_goal_percentage': item.get('v_field_goal_percentage'),
            'v_three_points_field_goals': item.get('v_three_points_field_goals'),
            'v_three_points_field_goal_attempts': item.get('v_three_points_field_goal_attempts'),
            'v_three_points_field_goal_percentage': item.get('v_three_points_field_goal_percentage'),
            'v_free_throws': item.get('v_free_throws'),
            'v_free_throw_attempts': item.get('v_free_throw_attempts'),
            'v_free_throw_percentage': item.get('v_free_throw_percentage'),
            'v_offensive_rebounds': item.get('v_offensive_rebounds'),
            'v_defensive_rebounds': item.get('v_defensive_rebounds'),
            'v_total_rebounds': item.get('v_total_rebounds'),
            'v_assists': item.get('v_assists'),
            'v_steals': item.get('v_steals'),
            'v_blocks': item.get('v_blocks'),
            'v_turn_over': item.get('v_turn_over'),
            'v_personnal_fouls': item.get('v_personnal_fouls'),
            'h_team_name': item.get('h_team_name'),
            'h_pts': item.get('h_pts'),
            'h_record': item.get('h_record'),
            'h_field_goals': item.get('h_field_goals'),
            'h_field_goal_attempts': item.get('h_field_goal_attempts'),
            'h_field_goal_percentage': item.get('h_field_goal_percentage'),
            'h_three_points_field_goals': item.get('h_three_points_field_goals'),
            'h_three_points_field_goal_attempts': item.get('h_three_points_field_goal_attempts'),
            'h_three_points_field_goal_percentage': item.get('h_three_points_field_goal_percentage'),
            'h_free_throws': item.get('h_free_throws'),
            'h_free_throw_attempts': item.get('h_free_throw_attempts'),
            'h_free_throw_percentage': item.get('h_free_throw_percentage'),
            'h_offensive_rebounds': item.get('h_offensive_rebounds'),
            'h_defensive_rebounds': item.get('h_defensive_rebounds'),
            'h_total_rebounds': item.get('h_total_rebounds'),
            'h_assists': item.get('h_assists'),
            'h_steals': item.get('h_steals'),
            'h_blocks': item.get('h_blocks'),
            'h_turn_over': item.get('h_turn_over'),
            'h_personnal_fouls': item.get('h_personnal_fouls'),
            'overtimes': item.get('overtimes'),
            'attendance': item.get('attendance'),
            'game_remarks': item.get('game_remarks')
        })

    keys = games[0].keys()
    with open('nba_games_data.csv', 'w', newline='', encoding='utf-8')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys, delimiter=';')
        dict_writer.writeheader()
        dict_writer.writerows(games)

In [6]:
def SaveInCsv_players(data):
    players = []
  
    for item in data:    
        players.append({
            'game_id':  item.get('game_id'),
            'player_name': item.get('player_name'),
            'minutes_played': item.get('minutes_played'),
            'field_goals': item.get('field_goals'),
            'field_goal_attempts': item.get('field_goal_attempts'),
            'field_goal_percentage': item.get('field_goal_percentage'),
            'three_points_field_goals': item.get('three_points_field_goals'),
            'three_points_field_goal_attempts': item.get('three_points_field_goal_attempts'),
            'three_points_field_goal_percentage': item.get('three_points_field_goal_percentage'),
            'free_throws': item.get('free_throws'),
            'free_throw_attempts': item.get('free_throw_attempts'),
            'free_throw_percentage': item.get('free_throw_percentage'),
            'offensive_rebounds': item.get('offensive_rebounds'),
            'defensive_rebounds': item.get('defensive_rebounds'),
            'total_rebounds': item.get('total_rebounds'),
            'assists': item.get('assists'),
            'steals': item.get('steals'),
            'blocks': item.get('blocks'),
            'turn_over': item.get('turn_over'),
            'personnal_fouls': item.get('personnal_fouls'),
            'points': item.get('points'),
            'plus_minus': item.get('plus_minus'),
        })

    keys = players[0].keys()
    with open('nba_player_stats_data.csv', 'w', newline='', encoding='UTF-8')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys, delimiter=';')
        dict_writer.writeheader()
        dict_writer.writerows(players)


#### Fonction pour récupérer les stats générales des games

In [7]:
def scrap_data(scrap_team_nb, scrap_player_nb):
    
#     browser.get("https://www.basketball-reference.com/"+link) 
#     html = browser.page_source

    soup = BeautifulSoup(html, 'lxml')
    schedule = soup.find(class_='table_container is_setup')

    schedule_sorted = soup.find(class_='table_container is_setup')
    tbody = schedule_sorted.find('tbody')

    # On cherche et retire les ligne _thead
    unwanted_thead = tbody.find_all(class_='thead')
    for unwanted in unwanted_thead:
        unwanted.extract()

    for row in tbody.find_all('tr'):
#         On décommente si on veut cut juste après le décompte - On laisse commenter pour finir l'année
        if (max_games <= len(games_list)):
            is_prev = False
            break;
        
        current_url = browser.current_url
        
        game_id = str(uuid.uuid4())
        
        match_stats = [] 
        

        try:
            game_box_score = row.find(attrs={"data-stat":'box_score_text'})

            link = game_box_score.find('a', href=True)
#     On recupére le nombre de joueurs, le record visiteur et home dans data
            data = navigate_on_box_score("https://www.basketball-reference.com/"+link['href'], current_url, game_id, scrap_player_nb)
            scrap_player_nb = data[0]
        
            visitor_record = data[1]
            stats_visitor = data[2]
            
            home_record = data[3]
            stats_home = data[4]
            
        except:
            print("impossible")
            visitor_record = ""
            home_record = ""
    
#         Liste des stats dans le tableau
        stats_match = ['date_game', 'game_start_time', 'visitor_team_name','visitor_pts', 
                 'home_team_name', 'home_pts', 'box_score_text', 
                 'overtimes', 'attendance', 'game_remarks']
    
        
        for stat in stats_match:
            try:
                match_data = row.find(attrs={"data-stat":stat}).text
                match_stats.append(match_data)
            except:
                match_stats.append("")
        
        new_game = {
            'game_id': game_id,
            'date_game': match_stats[0], 
            'game_start_time': match_stats[1],
            'v_team_name': match_stats[2],
            'v_pts': match_stats[3],
            'v_record': visitor_record,
            'v_field_goals': stats_visitor['field_goals'],
            'v_field_goal_attempts': stats_visitor['field_goal_attempts'],
            'v_field_goal_percentage': stats_visitor['field_goal_percentage'],
            'v_three_points_field_goals': stats_visitor['three_points_field_goals'],
            'v_three_points_field_goal_attempts': stats_visitor['three_points_field_goal_attempts'],
            'v_three_points_field_goal_percentage': stats_visitor['three_points_field_goal_percentage'],
            'v_free_throws': stats_visitor['free_throws'],
            'v_free_throw_attempts': stats_visitor['free_throw_attempts'],
            'v_free_throw_percentage': stats_visitor['free_throw_percentage'],
            'v_offensive_rebounds': stats_visitor['offensive_rebounds'],
            'v_defensive_rebounds': stats_visitor['defensive_rebounds'],
            'v_total_rebounds': stats_visitor['total_rebounds'],
            'v_assists': stats_visitor['assists'],
            'v_steals': stats_visitor['steals'],
            'v_blocks': stats_visitor['blocks'],
            'v_turn_over': stats_visitor['turn_over'],
            'v_personnal_fouls': stats_visitor['personnal_fouls'],
            'h_team_name': match_stats[4],
            'h_pts': match_stats[5],
            'h_record': home_record,
            'h_field_goals': stats_home['field_goals'],
            'h_field_goal_attempts': stats_home['field_goal_attempts'],
            'h_field_goal_percentage': stats_home['field_goal_percentage'],
            'h_three_points_field_goals': stats_home['three_points_field_goals'],
            'h_three_points_field_goal_attempts': stats_home['three_points_field_goal_attempts'],
            'h_three_points_field_goal_percentage': stats_home['three_points_field_goal_percentage'],
            'h_free_throws': stats_home['free_throws'],
            'h_free_throw_attempts': stats_home['free_throw_attempts'],
            'h_free_throw_percentage': stats_home['free_throw_percentage'],
            'h_offensive_rebounds': stats_home['offensive_rebounds'],
            'h_defensive_rebounds': stats_home['defensive_rebounds'],
            'h_total_rebounds': stats_home['total_rebounds'],
            'h_assists': stats_home['assists'],
            'h_steals': stats_home['steals'],
            'h_blocks': stats_home['blocks'],
            'h_turn_over': stats_home['turn_over'],
            'h_personnal_fouls': stats_home['personnal_fouls'],
            'box_score_text': match_stats[6],
            'overtimes': match_stats[7],
            'attendance': match_stats[8],
            'game_remarks': match_stats[9],
        }
        
        games_list.append(new_game)
        
        scrap_team_nb += 1
        print("Game #"+str(scrap_team_nb)+ " scrappée - "+str(scrap_player_nb)+" joueurs scrappés")
        
    return [scrap_team_nb, scrap_player_nb]

In [8]:
def navigate_on_box_score(url, url_back, game_id, scrap_player_nb):
    
    browser.get(url) 
    html = browser.page_source
    
#     On recupére le nb joueur - objet pour visiteur - objet home
    data = scrap_game_data(html, game_id, scrap_player_nb)
    
    scrap_player_nb = data[0]
    
    time.sleep(0.25)
    
    browser.get(url_back) 
    html = browser.page_source
    
    return data

#### Fonction pour récupérer les stats des joueurs dans la game

In [9]:
def scrap_game_data(html, game_id, scrap_player_nb):    
    soup = BeautifulSoup(html, 'lxml')
    
#     On récupère les records de team
    records = []
    teams_stats_list = []
    teams_record =  soup.findAll('div', id=re.compile("^box-...-game-basic_sh")) 
    
    for team_record in teams_record:
        try:
            record_data = team_record.find('h2').text
            
#             On récupère juste le score entre les parenthèses
            record_data_cleaned = record_data[record_data.find("(")+1:record_data.find(")")]
            records.append(record_data_cleaned)
        except:
            records.append("")
                
#     On récupère les stats des joueurs et des teams
    soup = BeautifulSoup(html, 'lxml')   
    stats_teams = soup.findAll('table', id=re.compile("^box-...-game-basic"))
    
    for team in stats_teams:
        
#         Liste des stats dans le tableau
        stats = ['player', 'mp', 'fg','fga', 
                 'fg_pct', 'fg3', 'fg3a', 
                 'fg3_pct', 'ft', 'fta', 'ft_pct',
                 'orb', 'drb', 'trb', 'ast', 'stl', 'blk',
                'tov', 'pf', 'pts', 'plus_minus']
        
        
#         On récupère les stats de l'équipe entière pour le match
        tfoot = team.find('tfoot')
        
        team_stats = []
        
        for stat in stats:
            try:
                team_data = tfoot.find(attrs={"data-stat":stat}).text
                team_stats.append(team_data)
            except:
                team_stats.append("")
                
  
        team_stats_item = {
            'field_goals': team_stats[2],
            'field_goal_attempts': team_stats[3],
            'field_goal_percentage': team_stats[4],
            'three_points_field_goals': team_stats[5],
            'three_points_field_goal_attempts': team_stats[6],
            'three_points_field_goal_percentage': team_stats[7],
            'free_throws': team_stats[8],
            'free_throw_attempts': team_stats[9],
            'free_throw_percentage': team_stats[10],
            'offensive_rebounds': team_stats[11],
            'defensive_rebounds': team_stats[12],
            'total_rebounds': team_stats[13],
            'assists': team_stats[14],
            'steals': team_stats[15],
            'blocks': team_stats[16],
            'turn_over': team_stats[17],
            'personnal_fouls': team_stats[18],
        }
        
        teams_stats_list.append(team_stats_item)
        
#         On récupère les stats de chaque jour pour le match
        tbody = team.find('tbody')
        
#         On cherche et retire les ligne _thead
        unwanted_thead = tbody.find_all(class_='thead')
        for unwanted in unwanted_thead:
            unwanted.extract()
        
        for row in tbody.find_all('tr') :

            player_stats = []            
            
            for stat in stats:
                try:
                    data = row.find(attrs={"data-stat":stat}).text
                    player_stats.append(data)
                except:
                    player_stats.append("")
                
  
            player_stats_item = {
                'game_id': game_id,
                'player_name': player_stats[0],
                'minutes_played': player_stats[1],
                'field_goals': player_stats[2],
                'field_goal_attempts': player_stats[3],
                'field_goal_percentage': player_stats[4],
                'three_points_field_goals': player_stats[5],
                'three_points_field_goal_attempts': player_stats[6],
                'three_points_field_goal_percentage': player_stats[7],
                'free_throws': player_stats[8],
                'free_throw_attempts': player_stats[9],
                'free_throw_percentage': player_stats[10],
                'offensive_rebounds': player_stats[11],
                'defensive_rebounds': player_stats[12],
                'total_rebounds': player_stats[13],
                'assists': player_stats[14],
                'steals': player_stats[15],
                'blocks': player_stats[16],
                'turn_over': player_stats[17],
                'personnal_fouls': player_stats[18],
                'points': player_stats[19],
                'plus_minus': player_stats[20],
            }

            players_stats_list.append(player_stats_item)

            scrap_player_nb += 1

#     data = [scrap_player_nb, home_record, visitor_home]
#     data = [scrap_player_nb, records[0], records[1]]
    
#     On va envoyer le nb joueur - objet stats team visitor - objet stats team home
    data = [scrap_player_nb, records[0], teams_stats_list[0], records[1], teams_stats_list[1]]
    
    return data

In [10]:
# scrap_game_data('https://www.basketball-reference.com/boxscores/201810160BOS.html')

In [11]:
html = None

# url = 'https://www.basketball-reference.com/leagues/NBA_2019_games.html'

url = 'https://www.basketball-reference.com/leagues/NBA_2021_games.html'


chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--start-maximized")

browser = webdriver.Chrome('C:/Users/beren/Desktop/chromedriver.exe', options=chrome_options)


browser.get(url)
html = browser.page_source

time.sleep(1)

cookie_button = browser.find_element_by_class_name("css-1gbvn3u")
cookie_button.click()

soup = BeautifulSoup(html, 'lxml')

is_prev = True
scrap_team_nb = 0
scrap_player_nb = 0

while is_prev == True:

    # On récupère les liens pour pouvoir naviguer de mois en mois
    link_pages = []
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    filter = soup.find(class_='filter')
    # print(filter)

    for link in filter.find_all('a'):
        link_pages.append(link.get('href'))
#     print(link_pages)

    for link in link_pages:
        
        browser.get("https://www.basketball-reference.com/"+link) 
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        
#         On décommente si on veut finir à la fin du mois après avoir atteint la limite de nombre de match
#         if (max_games <= len(games_list)):
#             is_prev = False
#             break;
            
        results = scrap_data(scrap_team_nb, scrap_player_nb)
        
        scrap_team_nb = results[0]
        scrap_player_nb = results[1]
    
    time.sleep(0.5)
    
#     Permet de cut à la fin de l'année après le nombre de match max indiqué
#     if (max_games <= len(games_list)):
#         is_prev = False
#         break;
    
    try:
        prev_button = browser.find_element_by_class_name("prev")
        prev_button.click()
    except:
        is_prev = False
    

browser.quit()


Game #1 scrappée - 28 joueurs scrappés
Game #2 scrappée - 56 joueurs scrappés
Game #3 scrappée - 83 joueurs scrappés
Game #4 scrappée - 112 joueurs scrappés
Game #5 scrappée - 141 joueurs scrappés
Game #6 scrappée - 170 joueurs scrappés
Game #7 scrappée - 199 joueurs scrappés
Game #8 scrappée - 228 joueurs scrappés
Game #9 scrappée - 255 joueurs scrappés
Game #10 scrappée - 281 joueurs scrappés
Game #11 scrappée - 309 joueurs scrappés
Game #12 scrappée - 338 joueurs scrappés
Game #13 scrappée - 364 joueurs scrappés
Game #14 scrappée - 392 joueurs scrappés
Game #15 scrappée - 421 joueurs scrappés


In [12]:
SaveInCsv(games_list)
SaveInCsv_players(players_stats_list)